# Temperament calculations in SageMath

## setup
Just run this once. Sets up basic functionality for temperament calculations.

In [82]:
# set p-limit module to use
# this should probably be a class instead of some globals, but its good enough for a notebook
def set_limit(p):
    n_dim = prime_pi(p);
    
    print("basis: ")
    print(prime_range(p+1))

    global log2
    def log2(x):
        return log(x)/log(2)

    # prime matrix
    global P
    P = matrix(RR, 1, n_dim, lambda i, j: log2(Primes().unrank(j)));

    # canonical map for n-edo
    global edo_map
    def edo_map(n):
        return matrix(ZZ, 1, n_dim, lambda i, j: round(n*log2(Primes().unrank(j))));

    # vector -> ratio
    global ratio
    def ratio(v):
        if not isinstance(v, list):
            v = v.list()
        res = 1/1;
        for i in range(len(v)):
            res *= Primes().unrank(i) ^ v[i]
        return res

    # ratio -> vector
    global factors
    def factors(x):
        F = list(x.factor())
        d = dict(F)

        solution = []
        for p in primes(1,stop = Primes().unrank(n_dim)):
            if p in d:
                solution.append(d[p])
            else:
                solution.append(0)
        assert (x == ratio(solution)), "Interval not contained in p-limit"
        return Matrix(n_dim,1,solution)

    global cents
    def cents(x):
        return 1200*x;

Use $ \mathbb{J}^{5} $:

In [83]:
set_limit(5)

basis: 
[2, 3, 5]


Some basic functionality

In [91]:
print("Vector form of 45/32:")
print(factors(45/32))
print("Ratio corresponding to [1 1 -1]:")
m3 = matrix([1,1,-1]).transpose()
print(ratio(m3))
print("Interval size:")
print("octaves:", P*m3)
print("cents:",cents(P*m3))
print("Get canonical 12-EDO map:")
print(edo_map(12
             ))

Vector form of 45/32:
[-5]
[ 2]
[ 1]
Ratio corresponding to [1 1 -1]:
6/5
Interval size:
octaves: [0.263034405833794]
cents: [315.641287000553]
Get canonical 12-EDO map:
[12 19 28]


Meantone map

In [3]:
T = matrix([[1, 1, 0], [0, 1, 4]])

In [3]:
#definte temperament from 7 & 12
T = block_matrix(ZZ, [[edo_map(19)], [edo_map(12)]],subdivide=False);
print(T);
print("\nHNF reduction:");
T = T.echelon_form();

#this is just to octave reduce the generator
T = copy(T).with_added_multiple_of_row(0,1,1)
print(T);

[19 30 44]
[12 19 28]

HNF reduction:
[1 1 0]
[0 1 4]


In [19]:
#calculate quarter comma meantone (just 2/1, 5/4)
J = block_matrix([[factors(2/1), factors(5/4)]],subdivide=False);

A = (T*J);
b = (P*J);

# wwe want to solve left: xA = b
gen = A.solve_left(b);
cents(gen)

#print(cents(P*factors(6/5)))  #Just minor third
#print(cents(gen*T*factors(6/5)))  #Factored through the temperament

[1200.00000000000 696.578428466209]

In [20]:
# calculate least squares optimal tuning wrt some intervals

J = block_matrix([[factors(2/1), factors(3/2), factors(5/4), factors(6/5)]], subdivide=False);

A = (T*J).transpose();
b = (P*J).transpose();

#OLS solve   (A^t A)^-1 A^t b
gen = (A.transpose() * A).inverse() * (A.transpose() * b);
gen = gen.transpose();
print(cents(gen))

[1203.39572993632 697.993315939677]


In [21]:
#calculate least squares optimal tuning wrt some intervals, with added constraint of pure octaves
J = block_matrix([[factors(3/2), factors(5/4), factors(5/3)]], subdivide=False);

A = (T*J).transpose();
b = (P*J).transpose();

# constraint matrix
# this says: 1*x + 0*y = 1
# aka the first generator is a just octave
C = matrix([1,0]);
d = matrix([1]);

E = block_matrix([[A.transpose() * A, C.transpose()], [C, 0]]);
f = block_matrix([[A.transpose() * b], [d]]);

#solve, the last number is the lagrange multiplier, which can be ignored
gen = (E.inverse()) * f;
gen = gen.transpose();
cents(gen)

[ 1200.00000000000  696.164845973964 0.827164984489093]

In [22]:
#find the nullspace (= tempered commas)
T = block_matrix(ZZ, [[edo_map(12)]],subdivide=False);
K = T.right_kernel().matrix();

def tenney(v):
    res = 0.0;
    for i in range(len(v)):
        res += log2(Primes().unrank(i)) * abs(v[i])
    return float(res)

# LLL reduction to get short commas
K = K.LLL();

commas = []
for x in range(0,10):
    for y in range(0,10):
        if x!=0 or y!=0 :
            v = x*K[0] + y*K[1]
            if P*v < 0:
                v = -v
            commas.append(v)

commas.sort(key = tenney)
for i in range(4):
    c = commas[i]
    print(c, ratio(c), cents(P*c))

#this operation is (usually) reversible
K.right_kernel().matrix()



(-4, 4, -1) 81/80 (21.5062895967156)
(7, 0, -3) 128/125 (41.0588584054956)
(3, 4, -4) 648/625 (62.5651480022107)
(11, -4, -2) 2048/2025 (19.5525688087805)


[12 19 28]

In [23]:
#find the temperament matrix corresponding to some commas
print("81/80 gives:")
print(block_matrix(ZZ, [[factors(81/80)]]).left_kernel().matrix())
print("81/80 and 128/125 gives:")
print(block_matrix(ZZ, [[factors(81/80), factors(128/125)]]).left_kernel().matrix())

81/80 gives:
[ 1  0 -4]
[ 0  1  4]
81/80 and 128/125 gives:
[12 19 28]


In [24]:
# sage has no troubles calculating the correct kernel when theres torsion
# (81/80)^3
T = block_matrix(ZZ, [[factors(81/80)*3]])

print("quotient group: ")
A = ZZ^3;
V = A.span(T.transpose());
M = A.quotient(V)
print(M)
print(M.optimized()[0].V())

print("mapping matrix from kernel: ")
K = T.left_kernel().matrix()
print(K)

quotient group: 
Finitely generated module V/W over Integer Ring with invariants (3, 0, 0)
Free module of degree 3 and rank 3 over Integer Ring
User basis matrix:
[ 4 -4  1]
[ 1 -1  0]
[ 0  1  0]
mapping matrix from kernel: 
[ 1  0 -4]
[ 0  1  4]


In [25]:
#removing contorsion by going to nullspace and back
T = block_matrix(ZZ, [[edo_map(5)], [edo_map(19)]],subdivide=False);
print("5&19 map: ")
print(T)
T = T.echelon_form();
print("HNF: ")
print(T)
K = T.right_kernel().matrix();

print("removing contorsion: ")
print(K.right_kernel().matrix())

5&19 map: 
[ 5  8 12]
[19 30 44]
HNF: 
[ 1  0 -4]
[ 0  2  8]
removing contorsion: 
[ 1  0 -4]
[ 0  1  4]


In [27]:
#some higher 11-limit temperaments
set_limit(11)

T1 = block_matrix(ZZ, [[edo_map(31)], [edo_map(22)]]).echelon_form()
print("Orwell:")
print(T1)

T2 = block_matrix(ZZ, [[edo_map(12)], [edo_map(15)]]).echelon_form()
print("Augene:")
print(T2)

print("Orwell & Augene:")
T = block_matrix(ZZ, [[T1], [T2]]).echelon_form()

print("comma:")
ratio(T.right_kernel().matrix()[0])

basis: 
[2, 3, 5, 7, 11]
Orwell:
[ 1  0  3  1  3]
[ 0  7 -3  8  2]
Augene:
[ 3  0  7 18 20]
[ 0  1  0 -2 -2]
Orwell & Augene:
comma:


176/175

In [29]:
## change of basis
set_limit(5)
T = block_matrix(ZZ, [[edo_map(19)], [edo_map(12)]]).echelon_form();

M = matrix([[1,2],[5,11]])
print(det(M))
T1 = M*T

#calculate quarter comma meantone (just 2/1, 5/4)
J = block_matrix([[factors(2/1), factors(5/4)]],subdivide=False);

A = (T*J);
b = (P*J);

# wwe want to solve left: xA = b
gen = A.solve_left(b);
print(cents(gen*(M.inverse())))

A = (T1*J);
b = (P*J);

# wwe want to solve left: xA = b
gen = A.solve_left(b);
print(cents(gen))

#print(cents(P*factors(6/5)))  #Just minor third
#print(cents(gen*T*factors(6/5)))  #Factored through the temperament

basis: 
[2, 3, 5]
1
[ 3717.10785766896 -503.421571533792]
[ 3717.10785766896 -503.421571533791]


In [30]:
# get unimodular matrix from HNF
T = block_matrix(ZZ, [[edo_map(19)], [edo_map(12)]]);
V,U = T.echelon_form(transformation=True)
U

[ 19 -30]
[-12  19]